<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/ufr_simulation_by_dc_wk_kitgrp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
import pandas as pd
import copy
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import ttest_1samp
import warnings
warnings.filterwarnings('ignore')

# Reading Data

In [2]:
data = pd.read_csv('analysis_5.csv')
data.head()

,FWK_ID,FISCAL_YEAR_PERIOD,REGION,PROD_ID,BASE_SKU,BARE_FLAG,KIT_SIZE,BATTERY_PACK,PLANT,DC,TOTAL_ORDER_QUANTITY,TOTAL_UFR_QUANTITY,TOTAL_MISS_QTY,UFR,AVG_WEEKLY_OH_QTY,PROJ_AVG_SFTY_STK_QTY,KIT_GRP
0,202439,2024009,US,DCK695P2,DCF887,N,Combo Kit (3+),P2,67,164,8,8,0,1.00000,1.0,11.0,combo
1,202442,2024010,US,DCK299M2,DCF887,N,2 Tool Kit,M2,67,164,21,21,0,1.00000,0.0,187.0,kit
2,202447,2024011,US,DCK486D2,DCF809,N,Combo Kit (3+),D2,768,164,278,74,204,0.26619,25.0,1654.0,combo
3,202449,2024012,US,DCK299D1W1,DCF887,N,2 Tool Kit,D1W1,67,166,232,232,0,1.00000,4.0,248.0,kit
4,202426,2024006,US,DCK551D1M1,DCF885,N,Combo Kit (3+),D1M1,67,164,46,44,2,0.95652,3.0,63.0,combo


# EDA

In [3]:
data.shape

(10035, 17)

In [4]:
data = data[data.TOTAL_ORDER_QUANTITY > 0]
data.fillna(0, inplace=True)
# Identify shortfalls
data['Total_Available'] = data['AVG_WEEKLY_OH_QTY'] #+ data['PROJ_AVG_SFTY_STK_QTY'] #exclude and check
data['Shortfall'] = data['TOTAL_ORDER_QUANTITY'] - data['Total_Available']
data['Shortfall'] = data['Shortfall'].clip(lower=0)

In [5]:
data.shape

(9787, 19)

In [6]:
data.head()

,FWK_ID,FISCAL_YEAR_PERIOD,REGION,PROD_ID,BASE_SKU,BARE_FLAG,KIT_SIZE,BATTERY_PACK,PLANT,DC,TOTAL_ORDER_QUANTITY,TOTAL_UFR_QUANTITY,TOTAL_MISS_QTY,UFR,AVG_WEEKLY_OH_QTY,PROJ_AVG_SFTY_STK_QTY,KIT_GRP,Total_Available,Shortfall
0,202439,2024009,US,DCK695P2,DCF887,N,Combo Kit (3+),P2,67,164,8,8,0,1.00000,1.0,11.0,combo,1.0,7.0
1,202442,2024010,US,DCK299M2,DCF887,N,2 Tool Kit,M2,67,164,21,21,0,1.00000,0.0,187.0,kit,0.0,21.0
2,202447,2024011,US,DCK486D2,DCF809,N,Combo Kit (3+),D2,768,164,278,74,204,0.26619,25.0,1654.0,combo,25.0,253.0
3,202449,2024012,US,DCK299D1W1,DCF887,N,2 Tool Kit,D1W1,67,166,232,232,0,1.00000,4.0,248.0,kit,4.0,228.0
4,202426,2024006,US,DCK551D1M1,DCF885,N,Combo Kit (3+),D1M1,67,164,46,44,2,0.95652,3.0,63.0,combo,3.0,43.0


In [7]:
data.tail()

,FWK_ID,FISCAL_YEAR_PERIOD,REGION,PROD_ID,BASE_SKU,BARE_FLAG,KIT_SIZE,BATTERY_PACK,PLANT,DC,TOTAL_ORDER_QUANTITY,TOTAL_UFR_QUANTITY,TOTAL_MISS_QTY,UFR,AVG_WEEKLY_OH_QTY,PROJ_AVG_SFTY_STK_QTY,KIT_GRP,Total_Available,Shortfall
10030,202429,2024007,US,DCK592L2,DCF885,N,Combo Kit (3+),L2,NO_SOURCE,164,4,4,0,1.0,0.0,0.0,combo,0.0,4.0
10031,202413,2024003,US,DCF809B,DCF809,Y,Single Tool,0,67,155,52,52,0,1.0,0.0,113.0,bare,0.0,52.0
10032,202403,2024001,US,DCK590L2,DCF885,N,Combo Kit (3+),L2,NO_SOURCE,164,1,0,1,0.0,0.0,0.0,combo,0.0,1.0
10033,202447,2024011,US,DCF840B,DCF840,Y,Single Tool,0,768,155,14,14,0,1.0,33.0,0.0,bare,33.0,0.0
10034,202442,2024010,US,DCK275C2,DCF840,N,2 Tool Kit,C2,768,164,1,0,1,0.0,0.0,0.0,kit,0.0,1.0


In [8]:
data['KIT_SIZE'].drop_duplicates()

,KIT_SIZE
0,Combo Kit (3+)
1,2 Tool Kit
6,Single Tool


In [9]:
data[['KIT_SIZE', 'BARE_FLAG']].drop_duplicates()

,KIT_SIZE,BARE_FLAG
0,Combo Kit (3+),N
1,2 Tool Kit,N
6,Single Tool,N
9,Single Tool,Y


In [10]:
data[['KIT_SIZE', 'BARE_FLAG', 'KIT_GRP']].drop_duplicates()

,KIT_SIZE,BARE_FLAG,KIT_GRP
0,Combo Kit (3+),N,combo
1,2 Tool Kit,N,kit
6,Single Tool,N,single
9,Single Tool,Y,bare


In [11]:
print(f"Number of bare SKUS: {len(data[data.KIT_GRP == 'bare'][['PROD_ID']].drop_duplicates())}")
print(f"Number of bare SKUS: {len(data[data.KIT_GRP == 'single'][['PROD_ID']].drop_duplicates())}")
print(f"Number of bare SKUS: {len(data[data.KIT_GRP == 'kit'][['PROD_ID']].drop_duplicates())}")
print(f"Number of bare SKUS: {len(data[data.KIT_GRP == 'combo'][['PROD_ID']].drop_duplicates())}")

Number of bare SKUS: 5
Number of bare SKUS: 18
Number of bare SKUS: 41
Number of bare SKUS: 56


In [12]:
data[data['TOTAL_MISS_QTY'] > 0]

,FWK_ID,FISCAL_YEAR_PERIOD,REGION,PROD_ID,BASE_SKU,BARE_FLAG,KIT_SIZE,BATTERY_PACK,PLANT,DC,TOTAL_ORDER_QUANTITY,TOTAL_UFR_QUANTITY,TOTAL_MISS_QTY,UFR,AVG_WEEKLY_OH_QTY,PROJ_AVG_SFTY_STK_QTY,KIT_GRP,Total_Available,Shortfall
2,202447,2024011,US,DCK486D2,DCF809,N,Combo Kit (3+),D2,768,164,278,74,204,0.26619,25.0,1654.0,combo,25.0,253.0
4,202426,2024006,US,DCK551D1M1,DCF885,N,Combo Kit (3+),D1M1,67,164,46,44,2,0.95652,3.0,63.0,combo,3.0,43.0
5,202451,2024012,US,DCK277C2,DCF787,N,2 Tool Kit,C2,NO_SOURCE,155,16,0,16,0.00000,0.0,0.0,kit,0.0,16.0
6,202417,2024004,US,DCF885C1,DCF885,N,Single Tool,C1,NO_SOURCE,155,1,0,1,0.00000,0.0,0.0,single,0.0,1.0
10,202430,2024007,US,DCF885C1,DCF885,N,Single Tool,C1,NO_SOURCE,164,2,0,2,0.00000,0.0,0.0,single,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10021,202431,2024007,US,DCK2100D1T1,DCF887,N,2 Tool Kit,D1T1,67,162,492,469,23,0.95325,1.0,1.0,kit,1.0,491.0
10023,202435,2024008,US,DCF809B,DCF809,Y,Single Tool,0,67,155,9,0,9,0.00000,0.0,113.0,bare,0.0,9.0
10025,202424,2024006,US,DCF809C2,DCF809,N,Single Tool,C2,67,155,36,35,1,0.97222,7.0,7.0,single,7.0,29.0
10032,202403,2024001,US,DCK590L2,DCF885,N,Combo Kit (3+),L2,NO_SOURCE,164,1,0,1,0.00000,0.0,0.0,combo,0.0,1.0


In [13]:
data[data.KIT_GRP == 'single'][['PROD_ID']].drop_duplicates()

,PROD_ID
6,DCF885C1
16,DCF840D1
18,DCF887P1
31,DCF787E1
46,DCF840E1
97,DCF887M2
112,DCF787C2
115,DCF787C1
119,DCF787D1
122,DCF887D1E1


# Helper Functions

In [14]:
def substitute_for_row_in_grp(idx, group_df):
    """
    For a given row within a group (rows sharing DC, FWK_ID, KIT_GRP, BATTERY_PACK, and SKU_GROUP),
    perform substitutions from other products that have available quantity (Total_Available > 0).
    Substitution is only allowed if TOTAL_MISS_QTY > 0 and candidates must have a different PROD_ID.

    This function updates the candidate's Total_Available in group_df (in place) and returns
    the remaining missing quantity for the given row.
    """
    row = group_df.loc[idx]
    missing = row['TOTAL_MISS_QTY']  # work only if missing exists
    if missing <= 0:
        return missing

    # Identify candidate substitute rows: same group, different index, positive available qty, and different PROD_ID.
    candidates = group_df[(group_df.index != idx) &
                          (group_df['Total_Available'] > 0) &
                          (group_df['PROD_ID'] != row['PROD_ID'])]

    # Iterate over candidates in sequence.
    for cand_idx, cand_row in candidates.iterrows():
        if missing <= 0:
            break
        available = cand_row['Total_Available']
        if available <= 0:
            continue  # skip if candidate has no available quantity

        sub_qty = min(missing, available)
        group_df.at[cand_idx, 'Total_Available'] = available - sub_qty
        missing -= sub_qty

    return missing


In [15]:
def substitute_for_row(idx, group_df):
    """
    For a given row (by index) within a group (all rows in same DC, WEEK, and KIT_GROUP),
    perform substitutions from other products with available qty > 0.
    Note that substitution is only allowed if MISS_QTY > 0.
    Only rows with a different PROD_ID can be used.

    This function updates the available qty in sim_data (the group_df copy) in place.
    """
    row = group_df.loc[idx]
    # Only try to substitute if there is a shortfall (MISS_QTY > 0)
    missing = row['TOTAL_MISS_QTY']
    if missing <= 0:
        return missing  # nothing to do if no shortage

    # Identify candidate substitute rows within the same group that are not this product
    candidates = group_df[(group_df.index != idx) & (group_df['Total_Available'] > 0)]
    # We iterate over candidates. (The order could be determined by a business rule like highest available first.)
    for cand_idx, cand_row in candidates.iterrows():
        if missing <= 0:
            break  # no more substitution needed
        # How many units can we take from the candidate
        available = cand_row['Total_Available']
        if available <= 0:
            continue #It does not have extra to provide for substitution

        # Determine substitution quantity: the minimum of what is missing and what is available
        sub_qty = min(missing, available)
        # Deduct the substitution quantity from the candidate available quantity
        group_df.at[cand_idx, 'Total_Available'] = available - sub_qty
        # Reduce the missing qty accordingly
        missing -= sub_qty

    return missing

In [16]:
def compute_cv(data, value_col):
    if data.empty:
        return 0, None, None, None
    count_val = len(data)
    mean_val = data[value_col].mean()
    std_val = data[value_col].std()
    # Only compute CV if mean is not zero to avoid division by zero
    cv = std_val / mean_val if mean_val != 0 else None
    return pd.Series({'Count': count_val, 'Mean': mean_val, 'Std': std_val, 'CV': cv})

In [17]:
def compute_metrics(df, col1, col2):
    if df.empty:
        return 0, None, None, None, None, None, None

    # Use the built-in 'int' instead of 'np.int'
    count = int(len(df)) #Changed from np.int to int
    total_order = df[col1].sum()
    total_ufr = df[col2].sum()

    # Calculate mean and standard deviation for demand (TOTAL_ORDER_QUANTITY)
    mean_demand = df[col1].mean()
    std_demand = df[col1].std()
    demand_cv = std_demand / mean_demand if mean_demand != 0 else None

    # Compute UFR rate substitution as the percentage of total order that is UFR quantity
    ufr_rate = ((total_ufr / total_order) * 100) if total_order != 0 else None

    return count, mean_demand, std_demand, demand_cv, total_order, total_ufr, ufr_rate

# Feature Engineering

In [18]:
data['DC'] = data['DC'].astype(str).str.strip()
data['FWK_ID'] = data['FWK_ID'].astype(str).str.strip()
data['KIT_GRP'] = data['KIT_GRP'].astype(str).str.strip()
data['PROD_ID'] = data['PROD_ID'].astype(str).str.strip()
data['Remaining_Shortfall'] = 0

# Variable Declaration

In [19]:
sim_grp_cols_1 = ['DC', 'FWK_ID', 'KIT_GRP', 'BATTERY_PACK']
sim_grp_cols_2 = ['DC', 'KIT_GRP', 'BATTERY_PACK']
sim_grp_cols_3 = ['DC', 'KIT_GRP']
sim_grp_cols_4 = ['DC', 'FWK_ID', 'KIT_GRP']
sim_grp_cols_5 = ['DC', 'FWK_ID', 'KIT_GRP', 'BATTERY_PACK', 'SKU_GRP']
unique_skus = sorted(data['BASE_SKU'].unique())
kit_grp = sorted(data['KIT_GRP'].unique())
kit_grp.remove('combo')
sim_grp_cols_5 = ['DC', 'FWK_ID', 'KIT_GRP', 'BATTERY_PACK', 'SKU_GROUP']

In [29]:
init_fact_agg = {'TOTAL_ORDER_QUANTITY' : 'sum',
            'TOTAL_MISS_QTY'       : 'sum',
            'TOTAL_UFR_QUANTITY'   : 'sum'
          }

In [20]:
fact_agg = {'TOTAL_ORDER_QUANTITY' : 'sum',
            'TOTAL_MISS_QTY'       : 'sum',
            'TOTAL_UFR_QUANTITY'   : 'sum',
            'REMAINING_MISS_QTY'   : 'sum'
          }

In [21]:
sel_cols = ['FWK_ID',
            'DC',
            'PROD_ID',
            'BATTERY_PACK',
            'KIT_GRP',
            'TOTAL_ORDER_QUANTITY',
            'TOTAL_UFR_QUANTITY',
            'TOTAL_MISS_QTY',
            'UFR',
            'AVG_WEEKLY_OH_QTY',
            'PROJ_AVG_SFTY_STK_QTY',
            'Shortfall',
            'REMAINING_MISS_QTY',
            'FILL_RATE_AFTER'
           ]

In [22]:
custom_order_dict = {'bare': 0, 'single': 1, 'kit': 2, 'combo': 3}

# Demand Variability

Variability by kit group does not support our assumption of bare and single tool has stable demand. In contrary, the results are higher for bare and single.

In [23]:
# Demand variability by KIT_GRP
cv_by_kit = data.groupby('KIT_GRP').apply(lambda x: compute_cv(x, 'TOTAL_ORDER_QUANTITY')).reset_index()
print("Demand variability by KIT_GRP:")
cv_by_kit.sort_values(by=['CV'], ascending=[True]).reset_index(drop=True)

Demand variability by KIT_GRP:


,KIT_GRP,Count,Mean,Std,CV
0,kit,3347.0,463.030475,1572.075150,3.395187
1,combo,4310.0,89.047332,308.661084,3.466259
2,single,1529.0,309.971223,1366.679135,4.409052
3,bare,601.0,41.876872,222.628932,5.316274


So, let's take a pick by base sku and kit group.

In [24]:
# Demand variability by KIT_GRP, Base SKU
cv_by_base_sku_kit = data.groupby(['BASE_SKU','KIT_GRP']).apply(lambda x: compute_cv(x, 'TOTAL_ORDER_QUANTITY')).reset_index()
print("Demand variability by Base SKU and KIT_GRP:")
cv_by_base_sku_kit.sort_values(by=['BASE_SKU','CV'], ascending=[True, True]).reset_index(drop=True)

Demand variability by Base SKU and KIT_GRP:


,BASE_SKU,KIT_GRP,Count,Mean,Std,CV
0,DCF787,bare,155.0,9.270968,16.627043,1.793453
1,DCF787,single,341.0,159.926686,334.876681,2.093939
2,DCF787,kit,330.0,283.836364,761.259617,2.682037
3,DCF787,combo,702.0,153.319088,413.042665,2.694007
4,DCF809,bare,139.0,34.007194,34.791823,1.023072
5,DCF809,combo,376.0,296.351064,692.265525,2.335964
6,DCF809,single,433.0,549.545035,1981.031003,3.604857
7,DCF809,kit,470.0,639.431915,2442.611773,3.819972
8,DCF840,bare,97.0,27.907216,58.652949,2.101713
9,DCF840,combo,238.0,194.651261,416.135963,2.137854


In [25]:
cv_by_base_sku_kit.to_csv('cv_by_base_sku_kit.csv', index=False)

For all but DCF887 base skus, bare sku has the lowest demand variability.

In [26]:
# Demand variability by KIT_GRP, Base SKU and DC
cv_by_base_sku_kit_dc = data.groupby(['DC', 'BASE_SKU', 'KIT_GRP']).apply(lambda x: compute_cv(x, 'TOTAL_ORDER_QUANTITY')).reset_index()
print("Demand variability by Base SKU and KIT_GRP and DC:")
cv_by_base_sku_kit_dc.sort_values(by=['DC','BASE_SKU','CV'], ascending=[True, True,True]).reset_index(drop=True)

Demand variability by Base SKU and KIT_GRP and DC:


,DC,BASE_SKU,KIT_GRP,Count,Mean,Std,CV
0,155,DCF787,bare,51.0,19.588235,25.377294,1.295538
1,155,DCF787,single,161.0,212.993789,427.706288,2.008069
2,155,DCF787,combo,291.0,196.281787,499.389237,2.544246
3,155,DCF787,kit,168.0,365.089286,960.754904,2.631561
4,155,DCF809,bare,52.0,42.038462,36.297227,0.863429
...,...,...,...,...,...,...,...
83,306,DCF885,kit,51.0,2.411765,1.458444,0.604721
84,306,DCF885,combo,121.0,1.933884,1.406743,0.727418
85,306,DCF887,single,27.0,1.666667,0.784465,0.470679
86,306,DCF887,kit,78.0,1.820513,0.936193,0.514247


In [27]:
cv_by_base_sku_kit_dc.to_csv('cv_by_base_sku_kit_dc.csv', index=False)

# UFR

## By DC X KIT Group

In [32]:
df_ufr_1 = data.groupby(['DC', 'KIT_GRP']).agg(init_fact_agg).reset_index()
df_ufr_1['UFR'] = df_ufr_1['TOTAL_UFR_QUANTITY'] / df_ufr_1['TOTAL_ORDER_QUANTITY']
df_ufr_1['KIT_GRP'] = pd.Categorical(df_ufr_1['KIT_GRP'], categories = custom_order_dict, ordered=True)
df_ufr_1.sort_values(by=['DC', 'KIT_GRP'], inplace = True)
df_ufr_1

,DC,KIT_GRP,TOTAL_ORDER_QUANTITY,TOTAL_MISS_QTY,TOTAL_UFR_QUANTITY,UFR
0,155,bare,15373,7072,8301,0.539973
3,155,single,242266,25882,216384,0.893167
2,155,kit,824625,153540,671085,0.813806
1,155,combo,206119,64085,142034,0.689087
4,162,bare,2507,172,2335,0.931392
7,162,single,71882,5966,65916,0.917003
6,162,kit,180165,28245,151920,0.843227
5,162,combo,51549,10244,41305,0.801276
8,164,bare,6487,2593,3894,0.600277
11,164,single,110002,8165,101837,0.925774


## DC X BASE SKU X KIT GRP

In [35]:
df_ufr_2 = data.groupby(['DC', 'BASE_SKU', 'KIT_GRP']).agg(init_fact_agg).reset_index()
df_ufr_2['UFR'] = df_ufr_2['TOTAL_UFR_QUANTITY'] / df_ufr_2['TOTAL_ORDER_QUANTITY']
df_ufr_2['KIT_GRP'] = pd.Categorical(df_ufr_2['KIT_GRP'], categories = custom_order_dict, ordered=True)
df_ufr_2.sort_values(by=['DC', 'KIT_GRP'], inplace = True)
df_ufr_2

,DC,BASE_SKU,KIT_GRP,TOTAL_ORDER_QUANTITY,TOTAL_MISS_QTY,TOTAL_UFR_QUANTITY,UFR
0,155,DCF787,bare,999,276,723,0.723724
4,155,DCF809,bare,2186,225,1961,0.897072
8,155,DCF840,bare,1844,59,1785,0.968004
12,155,DCF885,bare,983,628,355,0.361139
16,155,DCF887,bare,9361,5884,3477,0.371435
...,...,...,...,...,...,...,...
84,306,DCF885,kit,123,6,117,0.951220
86,306,DCF887,kit,142,2,140,0.985915
79,306,DCF787,combo,54,0,54,1.000000
81,306,DCF809,combo,1,0,1,1.000000


In [37]:
df_ufr_2[(df_ufr_2.DC == '155') & (df_ufr_2.BASE_SKU.isin(['DCF809', 'DCF840']))]

,DC,BASE_SKU,KIT_GRP,TOTAL_ORDER_QUANTITY,TOTAL_MISS_QTY,TOTAL_UFR_QUANTITY,UFR
4,155,DCF809,bare,2186,225,1961,0.897072
8,155,DCF840,bare,1844,59,1785,0.968004
7,155,DCF809,single,103294,11368,91926,0.889945
11,155,DCF840,single,100703,10196,90507,0.898752
6,155,DCF809,kit,143899,9585,134314,0.933391
10,155,DCF840,kit,180641,32796,147845,0.818447
5,155,DCF809,combo,54599,2450,52149,0.955127
9,155,DCF840,combo,28415,13229,15186,0.534436


In [38]:
df_ufr_2[(df_ufr_2.DC == '155') & (~df_ufr_2.BASE_SKU.isin(['DCF809', 'DCF840']))]

,DC,BASE_SKU,KIT_GRP,TOTAL_ORDER_QUANTITY,TOTAL_MISS_QTY,TOTAL_UFR_QUANTITY,UFR
0,155,DCF787,bare,999,276,723,0.723724
12,155,DCF885,bare,983,628,355,0.361139
16,155,DCF887,bare,9361,5884,3477,0.371435
3,155,DCF787,single,34292,2432,31860,0.929080
15,155,DCF885,single,819,449,370,0.451770
19,155,DCF887,single,3158,1437,1721,0.544965
2,155,DCF787,kit,61335,9782,51553,0.840515
14,155,DCF885,kit,260904,48618,212286,0.813656
18,155,DCF887,kit,177846,52759,125087,0.703344
1,155,DCF787,combo,57118,21934,35184,0.615988


This result shows that UFR for DCF840 and DCF809 single and kit are already great individually. Other than DCF787, UFR are nit so good at DC 155.

In [61]:
df_ufr_1.to_csv('df_ufr_1.csv', index=False)
df_ufr_2.to_csv('df_ufr_2.csv', index=False)

# Simulation

Here we are substituting an UFR cut in the same DC, week, kit group and battery pack. We have kept combo kit group out of equation as combo kit can comprise of multiple different types of tools and substitution for combo is difficult.
For now, we are focusing on bare. single and kit groups.
In the process, we assume all the kitgrp-battery packs are interchangable. in other words, we consider them as same product, just a different sku number.
In case of a cut, it searches from the pull where there is available inventory (on hand inv - total demand) to use for substitution. If used for substitution, it updates the total availabale. At the end of the process, we have a new updated remaining miss quantity which we started with original miss qunatity and left with updated miss qunatity after going thru the substitution process. The ratio of total order - remaining miss and total order qty is the simulated UFR.

In [39]:
# sim_grp_cols_1 = ['DC', 'FWK_ID', 'KIT_GRP', 'BATTERY_PACK']
sim_data = copy.deepcopy(data)
# We now simulate substitutions for each DC-week-kit group independently.
# Create a new column for the remaining missing qty after substitutions.
sim_data['REMAINING_MISS_QTY'] = sim_data['TOTAL_MISS_QTY']
# Process each group
grouped = sim_data[sim_data.KIT_GRP != 'combo'].groupby(sim_grp_cols_1)

In [40]:
for group_keys, group_df in grouped:
    # Work on a copy of the group that we update iteratively
    group_df = group_df.copy()

    # Process each row that has a missing qty > 0.
    for idx in group_df.index:
        if group_df.at[idx, 'TOTAL_MISS_QTY'] > 0:
            # Substitute if missing qty > 0
            rem = substitute_for_row(idx, group_df)
            # Update remaining missing quantity for that row.
            group_df.at[idx, 'REMAINING_MISS_QTY'] = rem
    # Update the original simulation data with the results from this group
    sim_data.update(group_df)

    # Now, calculate fill rates for each row
# Fill Rate = 1 - (Missing / Ordered).
sim_data['FILL_RATE_BEFORE'] = sim_data['UFR']
sim_data['FILL_RATE_AFTER'] = (sim_data['TOTAL_ORDER_QUANTITY'] - sim_data['REMAINING_MISS_QTY']) / sim_data['TOTAL_ORDER_QUANTITY']

# For overall or per-group analysis, you might average or otherwise aggregate the fill rates.
# Here we simply prepare paired series to run a paired t-test.
# Let’s work on rows where order qty > 0 to avoid division issues.
mask = data['TOTAL_ORDER_QUANTITY'] > 0
before = sim_data.loc[mask, 'FILL_RATE_BEFORE']
after = sim_data.loc[mask, 'FILL_RATE_AFTER']

# Print results (for debug / inspection)
'''
print("Before substitution fill rates:")
print(before)
print("After substitution fill rates:")
print(after)
'''
# Perform a paired t-test.
t_stat, p_val = stats.ttest_rel(before, after)
print(f"T-statistic: {t_stat}, P-value: {p_val}")

T-statistic: -31.127169346517427, P-value: 6.456674373195003e-203


Sim_data is at DC-PROD-WEEK-KITGRP-BATPK level.

In [41]:
sim_data[(sim_data['TOTAL_MISS_QTY'] > 0) &
         (sim_data['KIT_GRP'] != 'combo') &
         (sim_data['FILL_RATE_BEFORE'] > .5) &
         (sim_data['FILL_RATE_AFTER'] < 1) ][sel_cols].sort_values(by='FWK_ID', ascending=False)

,FWK_ID,DC,PROD_ID,BATTERY_PACK,KIT_GRP,TOTAL_ORDER_QUANTITY,TOTAL_UFR_QUANTITY,TOTAL_MISS_QTY,UFR,AVG_WEEKLY_OH_QTY,PROJ_AVG_SFTY_STK_QTY,Shortfall,REMAINING_MISS_QTY,FILL_RATE_AFTER
5470,202452,162,DCK2100P2,P2,kit,28,23,5,0.82143,24.0,222.0,4.0,4,0.857143
3775,202452,155,DCK240C2,C2,kit,586,524,62,0.89420,367.0,20669.0,219.0,61,0.895904
2119,202452,166,DCK2100D1T1,D1T1,kit,169,164,5,0.97041,13.0,2.0,156.0,5,0.970414
3142,202452,155,DCK2100P2,P2,kit,304,285,19,0.93750,136.0,1659.0,168.0,12,0.960526
1441,202452,162,DCK2100D1T1,D1T1,kit,28,18,10,0.64286,1.0,1.0,27.0,10,0.642857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3552,202401,164,DCK2100D1T1,D1T1,kit,60,45,15,0.75000,38.0,50.0,22.0,15,0.750000
2231,202401,155,DCF787C1,C1,single,14,12,2,0.85714,0.0,0.0,14.0,2,0.857143
8952,202401,155,DCF885M2,M2,single,66,48,18,0.72727,0.0,0.0,66.0,18,0.727273
9796,202401,155,DCK299D1W1,D1W1,kit,1012,972,40,0.96047,9.0,437.0,1003.0,40,0.960474


In [42]:
df_analysis_1 = sim_data[(sim_data['TOTAL_MISS_QTY'] > 0) & (sim_data.KIT_GRP != 'combo')]
df_analysis_1['UFR'] = df_analysis_1['UFR'].apply(lambda x: round(x, 5))
df_analysis_1['FILL_RATE_AFTER'] = df_analysis_1['FILL_RATE_AFTER'].apply(lambda x: round(x, 5))
df_analysis_1['UFR_CHG_VAL'] = 100 * (df_analysis_1[['UFR','FILL_RATE_AFTER']].pct_change(axis=1)['FILL_RATE_AFTER'])
df_analysis_1['UFR_CHG_VAL'] = df_analysis_1['UFR_CHG_VAL'].apply(lambda x: round(x, 5))
df_analysis_1[(df_analysis_1.TOTAL_MISS_QTY != df_analysis_1.REMAINING_MISS_QTY)]

,FWK_ID,FISCAL_YEAR_PERIOD,REGION,PROD_ID,BASE_SKU,BARE_FLAG,KIT_SIZE,BATTERY_PACK,PLANT,DC,...,AVG_WEEKLY_OH_QTY,PROJ_AVG_SFTY_STK_QTY,KIT_GRP,Total_Available,Shortfall,Remaining_Shortfall,REMAINING_MISS_QTY,FILL_RATE_BEFORE,FILL_RATE_AFTER,UFR_CHG_VAL
5,202451,2024012,US,DCK277C2,DCF787,N,2 Tool Kit,C2,NO_SOURCE,155,...,0.0,0.0,kit,0.0,16.0,0,0,0.00000,1.00000,inf
24,202414,2024004,US,DCK241C2,DCF885,N,2 Tool Kit,C2,67,155,...,46.0,1794.0,kit,0.0,1899.0,0,1577,0.00000,0.18920,inf
27,202428,2024007,US,DCK248D2,DCF887,N,2 Tool Kit,D2,768,164,...,19.0,104.0,kit,9.0,0.0,0,0,0.75000,1.00000,33.33333
30,202429,2024007,US,DCK280C2,DCF885,N,2 Tool Kit,C2,67,155,...,1.0,84.0,kit,0.0,10.0,0,0,0.90909,1.00000,10.00011
40,202448,2024011,US,DCK226D2,DCF809,N,2 Tool Kit,D2,768,155,...,0.0,410.0,kit,0.0,4.0,0,0,0.00000,1.00000,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,202433,2024008,US,DCK287D2,DCF887,N,2 Tool Kit,D2,NO_SOURCE,155,...,0.0,0.0,kit,0.0,10.0,0,0,0.00000,1.00000,inf
10009,202443,2024010,US,DCK249M2,DCF887,N,2 Tool Kit,M2,768,155,...,8.0,31.0,kit,0.0,14.0,0,0,0.86364,1.00000,15.78899
10010,202426,2024006,US,DCK287D2,DCF887,N,2 Tool Kit,D2,NO_SOURCE,155,...,0.0,0.0,kit,0.0,1.0,0,0,0.00000,1.00000,inf
10016,202412,2024003,US,DCK241C2,DCF885,N,2 Tool Kit,C2,67,155,...,46.0,1794.0,kit,0.0,614.0,0,360,0.00000,0.45455,inf


In [43]:
'''
sim_grp_cols_3 = ['DC', 'KIT_GRP']
sim_grp_cols_4 = ['DC', 'FWK_ID', 'KIT_GRP']
'''

"\nsim_grp_cols_3 = ['DC', 'KIT_GRP']\nsim_grp_cols_4 = ['DC', 'FWK_ID', 'KIT_GRP']\n"

Now we are evaluating results at DC and KIT Grp level(except combo) where there was a cut and substitution helped or not. We see with substitution, at all DC and kit-grp level, there is an improvement in UFR, which is promising 😃.

In [44]:
df_analysis_1a = df_analysis_1[(df_analysis_1.TOTAL_MISS_QTY != df_analysis_1.REMAINING_MISS_QTY)].groupby(sim_grp_cols_3).agg(fact_agg).reset_index()
df_analysis_1a['FILL_RATE_BEFORE'] = df_analysis_1a['TOTAL_UFR_QUANTITY'] / df_analysis_1a['TOTAL_ORDER_QUANTITY']
df_analysis_1a['FILL_RATE_AFTER'] = ( df_analysis_1a['TOTAL_ORDER_QUANTITY'] - df_analysis_1a['REMAINING_MISS_QTY']) / df_analysis_1a['TOTAL_ORDER_QUANTITY']
df_analysis_1a['UFR_CHG_VAL'] = 100 * (df_analysis_1a[['FILL_RATE_BEFORE', 'FILL_RATE_AFTER']].pct_change(axis=1)['FILL_RATE_AFTER'])
df_analysis_1a['UFR_CHG_VAL'] = df_analysis_1a['UFR_CHG_VAL'].apply(lambda x: round(x, 5))
df_analysis_1a['FILL_RATE_BEFORE'] = df_analysis_1a['FILL_RATE_BEFORE'].apply(lambda x: round(x, 5))
df_analysis_1a['FILL_RATE_AFTER'] = df_analysis_1a['FILL_RATE_AFTER'].apply(lambda x: round(x, 5))
df_analysis_1a

,DC,KIT_GRP,TOTAL_ORDER_QUANTITY,TOTAL_MISS_QTY,TOTAL_UFR_QUANTITY,REMAINING_MISS_QTY,FILL_RATE_BEFORE,FILL_RATE_AFTER,UFR_CHG_VAL
0,155,bare,10536,6570,3966,5212,0.37642,0.50532,34.24105
1,155,kit,444314,116007,328307,77939,0.73891,0.82459,11.59524
2,155,single,136548,21934,114614,10605,0.83937,0.92234,9.88448
3,162,bare,44,38,6,14,0.13636,0.68182,400.00000
4,162,kit,63395,17963,45432,13442,0.71665,0.78796,9.95114
5,162,single,12832,4235,8597,2857,0.66997,0.77735,16.02885
6,164,bare,1951,1696,255,1498,0.13070,0.23219,77.64706
7,164,kit,177040,44037,133003,30274,0.75126,0.82900,10.34789
8,164,single,35282,6289,28993,4332,0.82175,0.87722,6.74991
9,166,bare,81,12,69,9,0.85185,0.88889,4.34783


Now, let's evaluate results at DC, WEEK and KIT Grp level(except combo) where there was a cut and substitution helped or not. We see the same trend where with substitution, at all DC-WEEK and kit-grp level, there is an improvement in UFR, which is so good to see 😃.

In [45]:
df_analysis_2 = df_analysis_1[(df_analysis_1.TOTAL_MISS_QTY != df_analysis_1.REMAINING_MISS_QTY)].groupby(sim_grp_cols_4).agg(fact_agg).reset_index()
df_analysis_2['FILL_RATE_BEFORE'] = df_analysis_2['TOTAL_UFR_QUANTITY'] / df_analysis_2['TOTAL_ORDER_QUANTITY']
df_analysis_2['FILL_RATE_AFTER'] = ( df_analysis_2['TOTAL_ORDER_QUANTITY'] - df_analysis_2['REMAINING_MISS_QTY']) / df_analysis_2['TOTAL_ORDER_QUANTITY']
df_analysis_2['UFR_CHG_VAL'] = 100 * (df_analysis_2[['FILL_RATE_BEFORE', 'FILL_RATE_AFTER']].pct_change(axis=1)['FILL_RATE_AFTER'])
df_analysis_2['UFR_CHG_VAL'] = df_analysis_2['UFR_CHG_VAL'].apply(lambda x: round(x, 5))
df_analysis_2['FILL_RATE_BEFORE'] = df_analysis_2['FILL_RATE_BEFORE'].apply(lambda x: round(x, 5))
df_analysis_2['FILL_RATE_AFTER'] = df_analysis_2['FILL_RATE_AFTER'].apply(lambda x: round(x, 5))
df_analysis_2

,DC,FWK_ID,KIT_GRP,TOTAL_ORDER_QUANTITY,TOTAL_MISS_QTY,TOTAL_UFR_QUANTITY,REMAINING_MISS_QTY,FILL_RATE_BEFORE,FILL_RATE_AFTER,UFR_CHG_VAL
0,155,202401,kit,887,877,10,847,0.01127,0.04510,300.00000
1,155,202402,kit,1428,505,923,282,0.64636,0.80252,24.16035
2,155,202402,single,3,2,1,0,0.33333,1.00000,200.00000
3,155,202403,bare,153,75,78,39,0.50980,0.74510,46.15385
4,155,202403,kit,54,31,23,0,0.42593,1.00000,134.78261
...,...,...,...,...,...,...,...,...,...,...
380,166,202450,kit,4005,540,3465,17,0.86517,0.99576,15.09380
381,166,202450,single,390,267,123,258,0.31538,0.33846,7.31707
382,166,202451,kit,1578,154,1424,20,0.90241,0.98733,9.41011
383,166,202451,single,74,1,73,0,0.98649,1.00000,1.36986


In [46]:
sim_data.to_csv('sim_data.csv', index=False)
df_analysis_2.to_csv('df_analysis_2.csv', index=False)
df_analysis_1a.to_csv('df_analysis_1a.csv', index=False)

# Combination Demand Variability

5C1 + 5C2 + 5C3 + 5C4 + 5C5
= 5 + 10 + 10 + 5 + 1
= 31

Now, 5C1 and 5C4 are complimentatry to each other data grouping wise. Similarly, 5C2 and 5C3 are complimentary.

In [47]:
ctr = 0
for r in [1, 2, 3, 4, 5]:
    for combo in itertools.combinations(unique_skus, r):
        print(combo)
        ctr += 1
print(ctr)

('DCF787',)
('DCF809',)
('DCF840',)
('DCF885',)
('DCF887',)
('DCF787', 'DCF809')
('DCF787', 'DCF840')
('DCF787', 'DCF885')
('DCF787', 'DCF887')
('DCF809', 'DCF840')
('DCF809', 'DCF885')
('DCF809', 'DCF887')
('DCF840', 'DCF885')
('DCF840', 'DCF887')
('DCF885', 'DCF887')
('DCF787', 'DCF809', 'DCF840')
('DCF787', 'DCF809', 'DCF885')
('DCF787', 'DCF809', 'DCF887')
('DCF787', 'DCF840', 'DCF885')
('DCF787', 'DCF840', 'DCF887')
('DCF787', 'DCF885', 'DCF887')
('DCF809', 'DCF840', 'DCF885')
('DCF809', 'DCF840', 'DCF887')
('DCF809', 'DCF885', 'DCF887')
('DCF840', 'DCF885', 'DCF887')
('DCF787', 'DCF809', 'DCF840', 'DCF885')
('DCF787', 'DCF809', 'DCF840', 'DCF887')
('DCF787', 'DCF809', 'DCF885', 'DCF887')
('DCF787', 'DCF840', 'DCF885', 'DCF887')
('DCF809', 'DCF840', 'DCF885', 'DCF887')
('DCF787', 'DCF809', 'DCF840', 'DCF885', 'DCF887')
31


In [48]:
results = []

# We want combinations from the total of unique SKUs taking 1, 2, 3, or 4 at a time
for r in [1, 2, 3, 4, 5]:
    # For each combination of size r
    for combo in itertools.combinations(unique_skus, r):
        for sz in kit_grp:
            # Selected group where BASE_SKU is in the chosen combination
            selected = data[(data['BASE_SKU'].isin(combo)) & (data['KIT_GRP'] == sz)]
            # Remaining group: those rows not in the selected combination
            remaining = data[(~data['BASE_SKU'].isin(combo)) & (data['KIT_GRP'] == sz)]

            count_sel, mean_sel, std_sel, cv_sel, ord_qty_sel, ufr_qty_sel, ufr_sel  = compute_metrics(selected, 'TOTAL_ORDER_QUANTITY', 'TOTAL_UFR_QUANTITY')
            count_rem, mean_rem, std_rem, cv_rem, ord_qty_rem, ufr_qty_rem, ufr_rem = compute_metrics(remaining, 'TOTAL_ORDER_QUANTITY', 'TOTAL_UFR_QUANTITY')

            results.append({
                'Selected_Combination': ", ".join(combo) + '||' + sz,
                'Size_Selected': r,
                'Count_Selected': count_sel,
                'Mean_Selected': mean_sel,
                'Std_Selected': std_sel,
                'CV_Selected': cv_sel,
                'Total_Order_Selected': ord_qty_sel,
                'Total_UFR_Selected': ufr_qty_sel,
                'UFR_Selected': ufr_sel,
                'Count_Remaining': count_rem,
                'Mean_Remaining': mean_rem,
                'Std_Remaining': std_rem,
                'CV_Remaining': cv_rem,
                'Total_Order_Remaining': ord_qty_rem,
                'Total_UFR_Remaining': ufr_qty_rem,
                'UFR_Remaining': ufr_rem
               })

# Convert results to a DataFrame for easy inspection
results_data = pd.DataFrame(results)
#results_data["Other_Combination"] = results_data[["Selected_Combination", "Size_Selected"]].apply(lambda x, y: ', '.join([e for e in unique_skus if e != x.split("||")[0]]) + "||" + x.split("||")[1])
results_data["Remaining_Combination"] = results_data.apply(lambda row: ', '.join([e for e in unique_skus if e not in row['Selected_Combination'].split("||")[0].split(", ")]) + "||" + row['Selected_Combination'].split("||")[1] if row['Size_Selected'] < 5 else '', axis=1)
results_data = results_data[['Selected_Combination',
                             'Count_Selected',
                             'Mean_Selected',
                             'Std_Selected',
                             'CV_Selected',
                             'Total_Order_Selected',
                             'Total_UFR_Selected',
                             'UFR_Selected',
                             'Remaining_Combination',
                             'Count_Remaining',
                             'Mean_Remaining',
                             'Std_Remaining',
                             'CV_Remaining',
                             'Total_Order_Remaining',
                             'Total_UFR_Remaining',
                             'UFR_Remaining'
                             ]]

# Sort by CV of the selected group (you could also sort by the remaining group)
results_data_sorted = results_data.sort_values(by='CV_Selected', na_position='last')
#print("Demand Variability (CV) for Each BASE_SKU Selection and Its Complement:")
#print(results_data_sorted)

# If you want to see separately the ones with lowest CV in the selected group and in the remaining group:
lowest_selected_cv = results_data.sort_values(by='CV_Selected', na_position='last').head(5)
lowest_remaining_cv = results_data.sort_values(by='CV_Remaining', na_position='last').head(5)

#print("\nTop 5 groups with lowest CV in the Selected Group:")
#print(lowest_selected_cv)

#print("\nTop 5 groups with lowest CV in the Remaining Group:")
#print(lowest_remaining_cv)

In [49]:
results_data.sort_values(by=['UFR_Selected', 'UFR_Remaining'], ascending=[False, False], na_position='last')

,Selected_Combination,Count_Selected,Mean_Selected,Std_Selected,CV_Selected,Total_Order_Selected,Total_UFR_Selected,UFR_Selected,Remaining_Combination,Count_Remaining,Mean_Remaining,Std_Remaining,CV_Remaining,Total_Order_Remaining,Total_UFR_Remaining,UFR_Remaining
6,DCF840||bare,97,27.907216,58.652949,2.101713,2707,2630,97.155523,"DCF787, DCF809, DCF885, DCF887||bare",504,44.565476,241.702690,5.423541,22461.0,12577.0,55.994835
27,"DCF809, DCF840||bare",236,31.500000,46.100122,1.463496,7434,7029,94.552058,"DCF787, DCF885, DCF887||bare",365,48.586301,283.216002,5.829133,17734.0,8178.0,46.114808
3,DCF809||bare,139,34.007194,34.791823,1.023072,4727,4399,93.061138,"DCF787, DCF840, DCF885, DCF887||bare",462,44.244589,253.221934,5.723229,20441.0,10808.0,52.874126
2,DCF787||single,341,159.926686,334.876681,2.093939,54535,50484,92.571743,"DCF809, DCF840, DCF885, DCF887||single",1188,353.039562,1537.513899,4.355075,419411.0,373590.0,89.074917
4,DCF809||kit,470,639.431915,2442.611773,3.819972,300533,277884,92.463723,"DCF787, DCF840, DCF885, DCF887||kit",2877,434.212722,1377.110036,3.171510,1249230.0,1014076.0,81.176084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,"DCF787, DCF885, DCF887||bare",365,48.586301,283.216002,5.829133,17734,8178,46.114808,"DCF809, DCF840||bare",236,31.500000,46.100122,1.463496,7434.0,7029.0,94.552058
12,DCF887||bare,118,123.974576,490.095795,3.953196,14629,6357,43.454782,"DCF787, DCF809, DCF840, DCF885||bare",483,21.819876,36.605821,1.677637,10539.0,8850.0,83.973812
42,"DCF885, DCF887||bare",210,77.604762,370.812648,4.778220,16297,7027,43.118365,"DCF787, DCF809, DCF840||bare",391,22.687980,38.836605,1.711770,8871.0,8180.0,92.210574
9,DCF885||bare,92,18.130435,24.828297,1.369426,1668,670,40.167866,"DCF787, DCF809, DCF840, DCF887||bare",509,46.168959,241.472001,5.230181,23500.0,14537.0,61.859574


In [50]:
results_data

,Selected_Combination,Count_Selected,Mean_Selected,Std_Selected,CV_Selected,Total_Order_Selected,Total_UFR_Selected,UFR_Selected,Remaining_Combination,Count_Remaining,Mean_Remaining,Std_Remaining,CV_Remaining,Total_Order_Remaining,Total_UFR_Remaining,UFR_Remaining
0,DCF787||bare,155,9.270968,16.627043,1.793453,1437,1151,80.097425,"DCF809, DCF840, DCF885, DCF887||bare",446,53.208520,257.357154,4.836766,23731.0,14056.0,59.230542
1,DCF787||kit,330,283.836364,761.259617,2.682037,93666,77112,82.326565,"DCF809, DCF840, DCF885, DCF887||kit",3017,482.630759,1635.457161,3.388630,1456097.0,1214848.0,83.431804
2,DCF787||single,341,159.926686,334.876681,2.093939,54535,50484,92.571743,"DCF809, DCF840, DCF885, DCF887||single",1188,353.039562,1537.513899,4.355075,419411.0,373590.0,89.074917
3,DCF809||bare,139,34.007194,34.791823,1.023072,4727,4399,93.061138,"DCF787, DCF840, DCF885, DCF887||bare",462,44.244589,253.221934,5.723229,20441.0,10808.0,52.874126
4,DCF809||kit,470,639.431915,2442.611773,3.819972,300533,277884,92.463723,"DCF787, DCF840, DCF885, DCF887||kit",2877,434.212722,1377.110036,3.171510,1249230.0,1014076.0,81.176084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,"DCF809, DCF840, DCF885, DCF887||kit",3017,482.630759,1635.457161,3.388630,1456097,1214848,83.431804,DCF787||kit,330,283.836364,761.259617,2.682037,93666.0,77112.0,82.326565
89,"DCF809, DCF840, DCF885, DCF887||single",1188,353.039562,1537.513899,4.355075,419411,373590,89.074917,DCF787||single,341,159.926686,334.876681,2.093939,54535.0,50484.0,92.571743
90,"DCF787, DCF809, DCF840, DCF885, DCF887||bare",601,41.876872,222.628932,5.316274,25168,15207,60.421964,,0,NaN,NaN,NaN,NaN,NaN,NaN
91,"DCF787, DCF809, DCF840, DCF885, DCF887||kit",3347,463.030475,1572.075150,3.395187,1549763,1291960,83.365005,,0,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
results_data.to_csv('results_data.csv', index=False)

In [52]:
results_data[results_data['Selected_Combination'].isin(['DCF787, DCF809, DCF840, DCF885, DCF887||kit', 'DCF787, DCF809, DCF840, DCF885, DCF887||bare', 'DCF787, DCF809, DCF840, DCF885, DCF887||single'])]

,Selected_Combination,Count_Selected,Mean_Selected,Std_Selected,CV_Selected,Total_Order_Selected,Total_UFR_Selected,UFR_Selected,Remaining_Combination,Count_Remaining,Mean_Remaining,Std_Remaining,CV_Remaining,Total_Order_Remaining,Total_UFR_Remaining,UFR_Remaining
90,"DCF787, DCF809, DCF840, DCF885, DCF887||bare",601,41.876872,222.628932,5.316274,25168,15207,60.421964,,0,NaN,NaN,NaN,NaN,NaN,NaN
91,"DCF787, DCF809, DCF840, DCF885, DCF887||kit",3347,463.030475,1572.075150,3.395187,1549763,1291960,83.365005,,0,NaN,NaN,NaN,NaN,NaN,NaN
92,"DCF787, DCF809, DCF840, DCF885, DCF887||single",1529,309.971223,1366.679135,4.409052,473946,424074,89.477282,,0,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
results_data[results_data['Selected_Combination'].isin(['DCF809||kit', 'DCF809||bare', 'DCF809||single'])]

,Selected_Combination,Count_Selected,Mean_Selected,Std_Selected,CV_Selected,Total_Order_Selected,Total_UFR_Selected,UFR_Selected,Remaining_Combination,Count_Remaining,Mean_Remaining,Std_Remaining,CV_Remaining,Total_Order_Remaining,Total_UFR_Remaining,UFR_Remaining
3,DCF809||bare,139,34.007194,34.791823,1.023072,4727,4399,93.061138,"DCF787, DCF840, DCF885, DCF887||bare",462,44.244589,253.221934,5.723229,20441.0,10808.0,52.874126
4,DCF809||kit,470,639.431915,2442.611773,3.819972,300533,277884,92.463723,"DCF787, DCF840, DCF885, DCF887||kit",2877,434.212722,1377.110036,3.171510,1249230.0,1014076.0,81.176084
5,DCF809||single,433,549.545035,1981.031003,3.604857,237953,209071,87.862309,"DCF787, DCF840, DCF885, DCF887||single",1096,215.322080,1013.141382,4.705237,235993.0,215003.0,91.105668


In [54]:
df_grouped = data.groupby('BASE_SKU')['TOTAL_ORDER_QUANTITY'].agg(['mean', 'std'])

# Calculate the coefficient of variation (std / mean)
df_grouped['cv'] = df_grouped['std'] / df_grouped['mean']

df_grouped.sort_values(by='cv', ascending=False).reset_index()

,BASE_SKU,mean,std,cv
0,DCF885,220.981913,1136.925471,5.144880
1,DCF809,461.665021,1826.521678,3.956379
2,DCF840,499.879713,1635.438969,3.271665
3,DCF787,168.369110,483.474033,2.871513
4,DCF887,127.845940,346.669811,2.711622


In [55]:
lowest_selected_cv

,Selected_Combination,Count_Selected,Mean_Selected,Std_Selected,CV_Selected,Total_Order_Selected,Total_UFR_Selected,UFR_Selected,Remaining_Combination,Count_Remaining,Mean_Remaining,Std_Remaining,CV_Remaining,Total_Order_Remaining,Total_UFR_Remaining,UFR_Remaining
3,DCF809||bare,139,34.007194,34.791823,1.023072,4727,4399,93.061138,"DCF787, DCF840, DCF885, DCF887||bare",462,44.244589,253.221934,5.723229,20441.0,10808.0,52.874126
30,"DCF809, DCF885||bare",231,27.683983,32.106882,1.159764,6395,5069,79.265051,"DCF787, DCF840, DCF887||bare",370,50.737838,282.389656,5.565662,18773.0,10138.0,54.003090
9,DCF885||bare,92,18.130435,24.828297,1.369426,1668,670,40.167866,"DCF787, DCF809, DCF840, DCF887||bare",509,46.168959,241.472001,5.230181,23500.0,14537.0,61.859574
48,"DCF787, DCF809, DCF885||bare",386,20.290155,28.427146,1.401031,7832,6220,79.417773,"DCF840, DCF887||bare",215,80.632558,367.641282,4.559464,17336.0,8987.0,51.840102
15,"DCF787, DCF809||bare",294,20.965986,29.469696,1.405596,6164,5550,90.038936,"DCF840, DCF885, DCF887||bare",307,61.902280,309.078628,4.993009,19004.0,9657.0,50.815618


In [56]:
lowest_remaining_cv

,Selected_Combination,Count_Selected,Mean_Selected,Std_Selected,CV_Selected,Total_Order_Selected,Total_UFR_Selected,UFR_Selected,Remaining_Combination,Count_Remaining,Mean_Remaining,Std_Remaining,CV_Remaining,Total_Order_Remaining,Total_UFR_Remaining,UFR_Remaining
84,"DCF787, DCF840, DCF885, DCF887||bare",462,44.244589,253.221934,5.723229,20441,10808,52.874126,DCF809||bare,139,34.007194,34.791823,1.023072,4727.0,4399.0,93.061138
57,"DCF787, DCF840, DCF887||bare",370,50.737838,282.389656,5.565662,18773,10138,54.003090,"DCF809, DCF885||bare",231,27.683983,32.106882,1.159764,6395.0,5069.0,79.265051
78,"DCF787, DCF809, DCF840, DCF887||bare",509,46.168959,241.472001,5.230181,23500,14537,61.859574,DCF885||bare,92,18.130435,24.828297,1.369426,1668.0,670.0,40.167866
39,"DCF840, DCF887||bare",215,80.632558,367.641282,4.559464,17336,8987,51.840102,"DCF787, DCF809, DCF885||bare",386,20.290155,28.427146,1.401031,7832.0,6220.0,79.417773
72,"DCF840, DCF885, DCF887||bare",307,61.902280,309.078628,4.993009,19004,9657,50.815618,"DCF787, DCF809||bare",294,20.965986,29.469696,1.405596,6164.0,5550.0,90.038936


# Grouping and Substitution Simulation

In [57]:
df_grp = results_data[['Selected_Combination', 'Remaining_Combination']].drop_duplicates()
df_grp['KIT_SZ'] = df_grp['Selected_Combination'].apply(lambda x: x.split("||")[1])
df_grp['MATCH_BASE_SKU_LIST'] = df_grp['Selected_Combination'].apply(lambda x: x.split("||")[0].split(", "))
df_grp['NON_MATCH_BASE_SKU_LIST'] = df_grp['Remaining_Combination'].apply(lambda x: x.split("||")[0].split(", "))
df_grp['ID'] = np.arange(df_grp.shape[0])
df_grp

,Selected_Combination,Remaining_Combination,KIT_SZ,MATCH_BASE_SKU_LIST,NON_MATCH_BASE_SKU_LIST,ID
0,DCF787||bare,"DCF809, DCF840, DCF885, DCF887||bare",bare,[DCF787],"[DCF809, DCF840, DCF885, DCF887]",0
1,DCF787||kit,"DCF809, DCF840, DCF885, DCF887||kit",kit,[DCF787],"[DCF809, DCF840, DCF885, DCF887]",1
2,DCF787||single,"DCF809, DCF840, DCF885, DCF887||single",single,[DCF787],"[DCF809, DCF840, DCF885, DCF887]",2
3,DCF809||bare,"DCF787, DCF840, DCF885, DCF887||bare",bare,[DCF809],"[DCF787, DCF840, DCF885, DCF887]",3
4,DCF809||kit,"DCF787, DCF840, DCF885, DCF887||kit",kit,[DCF809],"[DCF787, DCF840, DCF885, DCF887]",4
...,...,...,...,...,...,...
88,"DCF809, DCF840, DCF885, DCF887||kit",DCF787||kit,kit,"[DCF809, DCF840, DCF885, DCF887]",[DCF787],88
89,"DCF809, DCF840, DCF885, DCF887||single",DCF787||single,single,"[DCF809, DCF840, DCF885, DCF887]",[DCF787],89
90,"DCF787, DCF809, DCF840, DCF885, DCF887||bare",,bare,"[DCF787, DCF809, DCF840, DCF885, DCF887]",[],90
91,"DCF787, DCF809, DCF840, DCF885, DCF887||kit",,kit,"[DCF787, DCF809, DCF840, DCF885, DCF887]",[],91


In [58]:
df_grp['MATCH_BASE_SKU_COUNT'] = df_grp['MATCH_BASE_SKU_LIST'].str.len()
df_grp['NON_MATCH_BASE_SKU_COUNT'] = 5 - df_grp['MATCH_BASE_SKU_COUNT']
df_grp[df_grp['NON_MATCH_BASE_SKU_COUNT'] > 0]

,Selected_Combination,Remaining_Combination,KIT_SZ,MATCH_BASE_SKU_LIST,NON_MATCH_BASE_SKU_LIST,ID,MATCH_BASE_SKU_COUNT,NON_MATCH_BASE_SKU_COUNT
0,DCF787||bare,"DCF809, DCF840, DCF885, DCF887||bare",bare,[DCF787],"[DCF809, DCF840, DCF885, DCF887]",0,1,4
1,DCF787||kit,"DCF809, DCF840, DCF885, DCF887||kit",kit,[DCF787],"[DCF809, DCF840, DCF885, DCF887]",1,1,4
2,DCF787||single,"DCF809, DCF840, DCF885, DCF887||single",single,[DCF787],"[DCF809, DCF840, DCF885, DCF887]",2,1,4
3,DCF809||bare,"DCF787, DCF840, DCF885, DCF887||bare",bare,[DCF809],"[DCF787, DCF840, DCF885, DCF887]",3,1,4
4,DCF809||kit,"DCF787, DCF840, DCF885, DCF887||kit",kit,[DCF809],"[DCF787, DCF840, DCF885, DCF887]",4,1,4
...,...,...,...,...,...,...,...,...
85,"DCF787, DCF840, DCF885, DCF887||kit",DCF809||kit,kit,"[DCF787, DCF840, DCF885, DCF887]",[DCF809],85,4,1
86,"DCF787, DCF840, DCF885, DCF887||single",DCF809||single,single,"[DCF787, DCF840, DCF885, DCF887]",[DCF809],86,4,1
87,"DCF809, DCF840, DCF885, DCF887||bare",DCF787||bare,bare,"[DCF809, DCF840, DCF885, DCF887]",[DCF787],87,4,1
88,"DCF809, DCF840, DCF885, DCF887||kit",DCF787||kit,kit,"[DCF809, DCF840, DCF885, DCF887]",[DCF787],88,4,1


In [59]:
df_grp.iloc[88]['MATCH_BASE_SKU_LIST'], sim_data[sim_data['BASE_SKU'].isin(df_grp.iloc[88]['MATCH_BASE_SKU_LIST'])]['BASE_SKU'].unique()

(['DCF809', 'DCF840', 'DCF885', 'DCF887'],
 array(['DCF887', 'DCF809', 'DCF885', 'DCF840'], dtype=object))

In [60]:
# Create a list of dfs, which are copy of data.
# It will have two extra columns, ID and sku group.
# ID will run thru 0..29, denoting possible match and remaining base sku combinations.
# sku group is set to 1 if base skus are matching with id specific matching sku set.
# otherwise it is set to 2 to denote a non-match.
data_df_list = [copy.deepcopy(data) for i in range(30)]
match_base_sku_list = df_grp['MATCH_BASE_SKU_LIST'].drop_duplicates().reset_index(drop=True).values.tolist()
for i in range(len(data_df_list)):
    data_df_list[i]['ID'] = i
    sku_group = match_base_sku_list[i]
    data_df_list[i]['SKU_GROUP'] = str(sku_group)
    for idx, row in data_df_list[i].iterrows():
        if row['BASE_SKU'] in sku_group:
            data_df_list[i].at[idx, 'IN_SKU_GROUP'] = 1
        else:
            data_df_list[i].at[idx, 'IN_SKU_GROUP'] = 0

In [114]:
# To store the iteration-level average fill rates
fill_rate_before_list = []
fill_rate_after_list  = []

# To store detailed simulation results from each iteration.
iteration_results = []

# Process each iteration using the corresponding data copy.
for i in range(30):
    df_iter = data_df_list[i]

    # Add an iteration indicator (so we can tell the source later)
    df_iter['iteration'] = i

    # Calculate baseline fill rate per row (before substitution):
    # FILL_RATE_BEFORE = (TOTAL_ORDER_QUANTITY - TOTAL_MISS_QTY) / TOTAL_ORDER_QUANTITY
    mask_order = df_iter['TOTAL_ORDER_QUANTITY'] > 0
    df_iter.loc[mask_order, 'FILL_RATE_BEFORE'] = (
        (df_iter.loc[mask_order, 'TOTAL_ORDER_QUANTITY'] - df_iter.loc[mask_order, 'TOTAL_MISS_QTY'])
        / df_iter.loc[mask_order, 'TOTAL_ORDER_QUANTITY']
    )

    # Create a column for remaining missing quantity; initialize as TOTAL_MISS_QTY.
    df_iter['REMAINING_MISS_QTY'] = df_iter['TOTAL_MISS_QTY']

    # Make a copy of rows where substitution should be applied (KIT_GRP is not "combo").
    df_iter_sub = df_iter[df_iter['KIT_GRP'] != 'combo'].copy()

    # Group by the simulation keys.
    grouped = df_iter_sub.groupby(sim_grp_cols_5) # Group by: ['DC', 'FWK_ID', 'KIT_GRP', 'BATTERY_PACK', 'SKU_GROUP']

    # Process each group.
    for group_keys, group_df in grouped:
        # Work on a copy for modifications.
        group_copy = group_df.copy()

        # For each row with missing qty > 0, perform substitution.
        for idx in group_copy.index:
            if group_copy.at[idx, 'TOTAL_MISS_QTY'] > 0:
                rem = substitute_for_row_in_grp(idx, group_copy)
                group_copy.at[idx, 'REMAINING_MISS_QTY'] = rem

        # Update the substituted group back.
        df_iter_sub.update(group_copy)

    # Update the main dataframe with substituted groups.
    df_iter.update(df_iter_sub)

    # Calculate the fill rate after substitution:
    # FILL_RATE_AFTER = (TOTAL_ORDER_QUANTITY - REMAINING_MISS_QTY) / TOTAL_ORDER_QUANTITY
    df_iter.loc[mask_order, 'FILL_RATE_AFTER'] = (
        (df_iter.loc[mask_order, 'TOTAL_ORDER_QUANTITY'] - df_iter.loc[mask_order, 'REMAINING_MISS_QTY'])
        / df_iter.loc[mask_order, 'TOTAL_ORDER_QUANTITY']
    )

    # Compute the average fill rates for the current iteration.
    avg_fill_before = df_iter.loc[mask_order, 'FILL_RATE_BEFORE'].mean()
    avg_fill_after  = df_iter.loc[mask_order, 'FILL_RATE_AFTER'].mean()

    fill_rate_before_list.append(avg_fill_before)
    fill_rate_after_list.append(avg_fill_after)

    # Append the entire iteration's data to the results list.
    iteration_results.append(df_iter.copy())

# Combine data from all iterations into one large dataframe.
combined_results_df = pd.concat(iteration_results, ignore_index=True)

In [118]:
combined_results_df.columns

Index(['FWK_ID', 'FISCAL_YEAR_PERIOD', 'REGION', 'PROD_ID', 'BASE_SKU',
       'BARE_FLAG', 'KIT_SIZE', 'BATTERY_PACK', 'PLANT', 'DC',
       'TOTAL_ORDER_QUANTITY', 'TOTAL_UFR_QUANTITY', 'TOTAL_MISS_QTY', 'UFR',
       'AVG_WEEKLY_OH_QTY', 'PROJ_AVG_SFTY_STK_QTY', 'KIT_GRP',
       'Total_Available', 'Shortfall', 'Remaining_Shortfall', 'ID',
       'SKU_GROUP', 'IN_SKU_GROUP', 'iteration', 'FILL_RATE_BEFORE',
       'REMAINING_MISS_QTY', 'FILL_RATE_AFTER', 'MATCH_BASE_SKU_LIST_GRP',
       'NON_MATCH_BASE_SKU_LIST_GRP'],
      dtype='object')

In [125]:
combined_results_df['SKU_GRP'] = combined_results_df['ID'].apply(lambda x: match_base_sku_list[x])
combined_results_df['IN_SKU_GROUP'] = combined_results_df['IN_SKU_GROUP'].astype(int)

In [126]:
combined_results_df[combined_results_df.ID == 25][['BASE_SKU', 'SKU_GROUP', 'IN_SKU_GROUP']].astype(str).drop_duplicates()

,BASE_SKU,SKU_GROUP,IN_SKU_GROUP
244675,DCF887,"['DCF787', 'DCF809', 'DCF840', 'DCF885']",0
244677,DCF809,"['DCF787', 'DCF809', 'DCF840', 'DCF885']",1
244679,DCF885,"['DCF787', 'DCF809', 'DCF840', 'DCF885']",1
244680,DCF787,"['DCF787', 'DCF809', 'DCF840', 'DCF885']",1
244690,DCF840,"['DCF787', 'DCF809', 'DCF840', 'DCF885']",1


In [127]:
combined_results_df.shape

(293610, 27)

In [129]:
combined_results_df.to_csv('combined_results_df.csv', index=False)

In [145]:
df_combined_results_dc_skugrp = combined_results_df.groupby(['DC', 'ID','IN_SKU_GROUP']).agg(fact_agg).reset_index()
df_combined_results_dc_skugrp['FILL_RATE_BEFORE'] = df_combined_results_dc_skugrp['TOTAL_UFR_QUANTITY'] / df_combined_results_dc_skugrp['TOTAL_ORDER_QUANTITY']
df_combined_results_dc_skugrp['FILL_RATE_AFTER'] = ( df_combined_results_dc_skugrp['TOTAL_ORDER_QUANTITY'] - df_combined_results_dc_skugrp['REMAINING_MISS_QTY']) / df_combined_results_dc_skugrp['TOTAL_ORDER_QUANTITY']
df_combined_results_dc_skugrp['UFR_CHG_VAL'] = 100 * (df_combined_results_dc_skugrp[['FILL_RATE_BEFORE', 'FILL_RATE_AFTER']].pct_change(axis=1)['FILL_RATE_AFTER'])
for col in ['FILL_RATE_BEFORE', 'FILL_RATE_AFTER', 'UFR_CHG_VAL']:
    df_combined_results_dc_skugrp[col] = np.round(df_combined_results_dc_skugrp[col], 5)
df_combined_results_dc_skugrp[(df_combined_results_dc_skugrp.DC == '155') & (df_combined_results_dc_skugrp.TOTAL_MISS_QTY > df_combined_results_dc_skugrp.REMAINING_MISS_QTY)].sort_values(by = ['DC', 'ID'], ascending = [True, True]).iloc[1::2]

,DC,ID,IN_SKU_GROUP,TOTAL_ORDER_QUANTITY,TOTAL_MISS_QTY,TOTAL_UFR_QUANTITY,REMAINING_MISS_QTY,FILL_RATE_BEFORE,FILL_RATE_AFTER,UFR_CHG_VAL
1,155,0,1,153744,34424,119320,25200,0.77610,0.83609,7.73047
3,155,1,1,303978,23628,280350,7565,0.92227,0.97511,5.72962
5,155,2,1,311603,56280,255323,43828,0.81939,0.85935,4.87696
7,155,3,1,298562,61093,237469,54064,0.79538,0.81892,2.95997
9,155,4,1,220496,75154,145342,69410,0.65916,0.68521,3.95206
11,155,5,1,457722,58052,399670,32765,0.87317,0.92842,6.32697
13,155,6,1,465347,90704,374643,69028,0.80508,0.85166,5.78577
15,155,7,1,452306,95517,356789,79264,0.78882,0.82476,4.55535
17,155,8,1,374240,109578,264662,94610,0.70720,0.74719,5.65552
19,155,9,1,615581,79908,535673,51393,0.87019,0.91651,5.32321


In [150]:
combined_results_df.groupby(['DC', 'SKU_GROUP','IN_SKU_GROUP']).agg(fact_agg).reset_index()

,DC,SKU_GROUP,IN_SKU_GROUP,TOTAL_ORDER_QUANTITY,TOTAL_MISS_QTY,TOTAL_UFR_QUANTITY,REMAINING_MISS_QTY
0,155,"['DCF787', 'DCF809', 'DCF840', 'DCF885']",0,220496,75154,145342,69410
1,155,"['DCF787', 'DCF809', 'DCF840', 'DCF885']",1,1067887,175425,892462,130657
2,155,"['DCF787', 'DCF809', 'DCF840', 'DCF887']",0,298562,61093,237469,54064
3,155,"['DCF787', 'DCF809', 'DCF840', 'DCF887']",1,989821,189486,800335,146003
4,155,"['DCF787', 'DCF809', 'DCF840']",0,519058,136247,382811,123474
...,...,...,...,...,...,...,...
295,306,"['DCF885', 'DCF887']",1,730,8,722,8
296,306,['DCF885'],0,664,3,661,3
297,306,['DCF885'],1,357,6,351,6
298,306,['DCF887'],0,648,7,641,7


In [152]:
# Slice and dice at DC-sku group level
df_combined_results_dc_skugrp = combined_results_df.groupby(['DC', 'SKU_GROUP','IN_SKU_GROUP']).agg(fact_agg).reset_index()
df_combined_results_dc_skugrp['FILL_RATE_BEFORE'] = df_combined_results_dc_skugrp['TOTAL_UFR_QUANTITY'] / df_combined_results_dc_skugrp['TOTAL_ORDER_QUANTITY']
df_combined_results_dc_skugrp['FILL_RATE_AFTER'] = ( df_combined_results_dc_skugrp['TOTAL_ORDER_QUANTITY'] - df_combined_results_dc_skugrp['REMAINING_MISS_QTY']) / df_combined_results_dc_skugrp['TOTAL_ORDER_QUANTITY']
df_combined_results_dc_skugrp['UFR_CHG_VAL'] = 100 * (df_combined_results_dc_skugrp[['FILL_RATE_BEFORE', 'FILL_RATE_AFTER']].pct_change(axis=1)['FILL_RATE_AFTER'])
for col in ['FILL_RATE_BEFORE', 'FILL_RATE_AFTER', 'UFR_CHG_VAL']:
    df_combined_results_dc_skugrp[col] = np.round(df_combined_results_dc_skugrp[col], 5)
df_combined_results_dc_skugrp[(df_combined_results_dc_skugrp.DC == '155') & (df_combined_results_dc_skugrp.TOTAL_MISS_QTY > df_combined_results_dc_skugrp.REMAINING_MISS_QTY)]#.sort_values(by = ['DC', 'SKU_GROUP'], ascending = [True, True]).iloc[1::2]

,DC,SKU_GROUP,IN_SKU_GROUP,TOTAL_ORDER_QUANTITY,TOTAL_MISS_QTY,TOTAL_UFR_QUANTITY,REMAINING_MISS_QTY,FILL_RATE_BEFORE,FILL_RATE_AFTER,UFR_CHG_VAL
0,155,"['DCF787', 'DCF809', 'DCF840', 'DCF885']",0,220496,75154,145342,69410,0.65916,0.68521,3.95206
1,155,"['DCF787', 'DCF809', 'DCF840', 'DCF885']",1,1067887,175425,892462,130657,0.83573,0.87765,5.01624
2,155,"['DCF787', 'DCF809', 'DCF840', 'DCF887']",0,298562,61093,237469,54064,0.79538,0.81892,2.95997
3,155,"['DCF787', 'DCF809', 'DCF840', 'DCF887']",1,989821,189486,800335,146003,0.80857,0.85250,5.43310
4,155,"['DCF787', 'DCF809', 'DCF840']",0,519058,136247,382811,123474,0.73751,0.76212,3.33663
5,155,"['DCF787', 'DCF809', 'DCF840']",1,769325,114332,654993,76593,0.85139,0.90044,5.76174
6,155,"['DCF787', 'DCF809', 'DCF885', 'DCF887']",0,311603,56280,255323,43828,0.81939,0.85935,4.87696
7,155,"['DCF787', 'DCF809', 'DCF885', 'DCF887']",1,976780,194299,782481,156239,0.80108,0.84005,4.86402
8,155,"['DCF787', 'DCF809', 'DCF885']",0,532099,131434,400665,113238,0.75299,0.78719,4.54145
9,155,"['DCF787', 'DCF809', 'DCF885']",1,756284,119145,637139,86829,0.84246,0.88519,5.07205


In [154]:
# Slice and dice at DC-kitgrp-sku group level
df_combined_results_dc_kitgrp_skugrp = combined_results_df.groupby(['DC', 'KIT_GRP', 'SKU_GROUP', 'IN_SKU_GROUP']).agg(fact_agg).reset_index()
df_combined_results_dc_kitgrp_skugrp['FILL_RATE_BEFORE'] = df_combined_results_dc_kitgrp_skugrp['TOTAL_UFR_QUANTITY'] / df_combined_results_dc_kitgrp_skugrp['TOTAL_ORDER_QUANTITY']
df_combined_results_dc_kitgrp_skugrp['FILL_RATE_AFTER'] = ( df_combined_results_dc_kitgrp_skugrp['TOTAL_ORDER_QUANTITY'] - df_combined_results_dc_kitgrp_skugrp['REMAINING_MISS_QTY']) / df_combined_results_dc_kitgrp_skugrp['TOTAL_ORDER_QUANTITY']
df_combined_results_dc_kitgrp_skugrp['UFR_CHG_VAL'] = 100 * (df_combined_results_dc_kitgrp_skugrp[['FILL_RATE_BEFORE', 'FILL_RATE_AFTER']].pct_change(axis=1)['FILL_RATE_AFTER'])
for col in ['FILL_RATE_BEFORE', 'FILL_RATE_AFTER', 'UFR_CHG_VAL']:
    df_combined_results_dc_kitgrp_skugrp[col] = np.round(df_combined_results_dc_kitgrp_skugrp[col], 5)
df_combined_results_dc_kitgrp_skugrp[(df_combined_results_dc_kitgrp_skugrp.DC == '155') & (df_combined_results_dc_kitgrp_skugrp.TOTAL_MISS_QTY > df_combined_results_dc_kitgrp_skugrp.REMAINING_MISS_QTY)]#.sort_values(by = ['DC', 'SKU_GROUP'], ascending = [True, True]).iloc[1::2]

,DC,KIT_GRP,SKU_GROUP,IN_SKU_GROUP,TOTAL_ORDER_QUANTITY,TOTAL_MISS_QTY,TOTAL_UFR_QUANTITY,REMAINING_MISS_QTY,FILL_RATE_BEFORE,FILL_RATE_AFTER,UFR_CHG_VAL
0,155,bare,"['DCF787', 'DCF809', 'DCF840', 'DCF885']",0,9361,5884,3477,5064,0.37143,0.45903,23.58355
1,155,bare,"['DCF787', 'DCF809', 'DCF840', 'DCF885']",1,6012,1188,4824,650,0.80240,0.89188,11.15257
2,155,bare,"['DCF787', 'DCF809', 'DCF840', 'DCF887']",0,983,628,355,262,0.36114,0.73347,103.09859
3,155,bare,"['DCF787', 'DCF809', 'DCF840', 'DCF887']",1,14390,6444,7946,5452,0.55219,0.62113,12.48427
4,155,bare,"['DCF787', 'DCF809', 'DCF840']",0,10344,6512,3832,5326,0.37046,0.48511,30.94990
...,...,...,...,...,...,...,...,...,...,...,...
234,155,single,"['DCF885', 'DCF887']",0,238289,23996,214293,12748,0.89930,0.94650,5.24889
235,155,single,"['DCF885', 'DCF887']",1,3977,1886,2091,1805,0.52577,0.54614,3.87374
236,155,single,['DCF885'],0,241447,25433,216014,14185,0.89466,0.94125,5.20707
237,155,single,['DCF885'],1,819,449,370,368,0.45177,0.55067,21.89189


In [155]:
# Convert lists to numpy arrays.
before_arr = np.array(fill_rate_before_list)
after_arr  = np.array(fill_rate_after_list)

# Perform a paired t-test on the 30 paired iteration-level fill rates.
t_stat, p_val = stats.ttest_rel(before_arr, after_arr)
'''
print("Iteration-level average fill rates BEFORE substitution:")
print(before_arr)
print("Iteration-level average fill rates AFTER substitution:")
print(after_arr)
'''
print("\nPaired t-test results (comparing BEFORE vs. AFTER fill rates):")
print(f"T-statistic: {t_stat:.4f}, P-value: {p_val:.4f}")


Paired t-test results (comparing BEFORE vs. AFTER fill rates):
T-statistic: -15597577537306224.0000, P-value: 0.0000


In [156]:
combined_results_df = combined_results_df.sort_values(by=['ID', 'FWK_ID'], ascending=[True, True])

In [157]:
combined_results_df[(combined_results_df['ID'] == 29) &
                    (combined_results_df['KIT_GRP'] != 'combo') &
                    (combined_results_df['TOTAL_MISS_QTY'] > 0) &
                    (combined_results_df['FILL_RATE_BEFORE'] > .5) &
                    (combined_results_df['FILL_RATE_BEFORE'] < .9) &
                    (combined_results_df['FILL_RATE_AFTER'] < 1) &
                    (combined_results_df['FILL_RATE_BEFORE'] < combined_results_df['FILL_RATE_AFTER']) &
                    (combined_results_df['DC'] == '155')
                   ].sort_values(by='FILL_RATE_AFTER', ascending=False)

,FWK_ID,FISCAL_YEAR_PERIOD,REGION,PROD_ID,BASE_SKU,BARE_FLAG,KIT_SIZE,BATTERY_PACK,PLANT,DC,...,Total_Available,Shortfall,Remaining_Shortfall,ID,SKU_GROUP,IN_SKU_GROUP,iteration,FILL_RATE_BEFORE,REMAINING_MISS_QTY,FILL_RATE_AFTER
288872,202445,2024011,US,DCF809D1,DCF809,N,Single Tool,D1,67,155,...,458.0,3928.0,0,29,"['DCF809', 'DCF840', 'DCF885', 'DCF887']",1,29,0.877109,50,0.988600
287570,202410,2024003,US,DCK227D2,DCF840,N,2 Tool Kit,D2,67,155,...,1292.0,3504.0,0,29,"['DCF809', 'DCF840', 'DCF885', 'DCF887']",1,29,0.794834,128,0.973339
285620,202435,2024008,US,DCK299M2,DCF887,N,2 Tool Kit,M2,67,155,...,6.0,133.0,0,29,"['DCF809', 'DCF840', 'DCF885', 'DCF887']",1,29,0.892086,7,0.949640
288956,202443,2024010,US,DCF809D1,DCF809,N,Single Tool,D1,67,155,...,458.0,3642.0,0,29,"['DCF809', 'DCF840', 'DCF885', 'DCF887']",1,29,0.835610,215,0.947561
289063,202448,2024011,US,DCF840D1,DCF840,N,Single Tool,D1,768,155,...,0.0,1293.0,0,29,"['DCF809', 'DCF840', 'DCF885', 'DCF887']",1,29,0.522222,154,0.904938
293514,202431,2024007,US,DCK249M2,DCF887,N,2 Tool Kit,M2,768,155,...,8.0,23.0,0,29,"['DCF809', 'DCF840', 'DCF885', 'DCF887']",1,29,0.709677,3,0.903226
289863,202437,2024009,US,DCK241C2,DCF885,N,2 Tool Kit,C2,67,155,...,15.0,6304.0,0,29,"['DCF809', 'DCF840', 'DCF885', 'DCF887']",1,29,0.840000,649,0.897795
286396,202416,2024004,US,DCK228E2,DCF840,N,2 Tool Kit,E2,67,155,...,106.0,0.0,0,29,"['DCF809', 'DCF840', 'DCF885', 'DCF887']",1,29,0.827586,3,0.896552
287516,202452,2024012,US,DCK240C2,DCF885,N,2 Tool Kit,C2,67,155,...,301.0,219.0,0,29,"['DCF809', 'DCF840', 'DCF885', 'DCF887']",1,29,0.894198,61,0.895904
287307,202445,2024011,US,DCF787B,DCF787,Y,Single Tool,0,768,155,...,0.0,85.0,0,29,"['DCF809', 'DCF840', 'DCF885', 'DCF887']",0,29,0.659091,12,0.863636


In [166]:
sim_data_dc_kitgrp = sim_data.groupby(['DC', 'KIT_GRP']).agg({"TOTAL_ORDER_QUANTITY": 'sum',
                                                                              "TOTAL_UFR_QUANTITY": 'sum',
                                                                              "TOTAL_MISS_QTY": 'sum',
                                                                              "REMAINING_MISS_QTY": 'sum'
                                                                             }).reset_index()
sim_data_dc_kitgrp['FILL_RATE_BEFORE'] = sim_data_dc_kitgrp['TOTAL_UFR_QUANTITY'] / sim_data_dc_kitgrp['TOTAL_ORDER_QUANTITY']
sim_data_dc_kitgrp['FILL_RATE_AFTER'] = (sim_data_dc_kitgrp['TOTAL_ORDER_QUANTITY'] - sim_data_dc_kitgrp['REMAINING_MISS_QTY']) / sim_data_dc_kitgrp['TOTAL_ORDER_QUANTITY']
sim_data_dc_kitgrp['UFR_CHG_VAL'] = 100 * (sim_data_dc_kitgrp[['FILL_RATE_BEFORE', 'FILL_RATE_AFTER']].pct_change(axis=1)['FILL_RATE_AFTER'])
sim_data_dc_kitgrp[sim_data_dc_kitgrp.DC == '155']

,DC,KIT_GRP,TOTAL_ORDER_QUANTITY,TOTAL_UFR_QUANTITY,TOTAL_MISS_QTY,REMAINING_MISS_QTY,FILL_RATE_BEFORE,FILL_RATE_AFTER,UFR_CHG_VAL
0,155,bare,15373,8301,7072,5714,0.539973,0.628309,16.359475
1,155,combo,206119,142034,64085,64085,0.689087,0.689087,0.000000
2,155,kit,824625,671085,153540,115472,0.813806,0.859970,5.672605
3,155,single,242266,216384,25882,14553,0.893167,0.939930,5.235600


Previously, we have seen for DC 155 DCF809 and DCF840, single tool, UFR was ~90%. With substitution, we achive 93.4% UFR.

In [165]:
df_combined_results_dc_kitgrp_skugrp[(df_combined_results_dc_kitgrp_skugrp.DC == '155') &
                                      (df_combined_results_dc_kitgrp_skugrp.KIT_GRP == 'single') &
                                      (df_combined_results_dc_kitgrp_skugrp.IN_SKU_GROUP == 1) &
                                      (df_combined_results_dc_kitgrp_skugrp.TOTAL_ORDER_QUANTITY > 200000)
                                    ].sort_values(by='UFR_CHG_VAL', ascending=False)

,DC,KIT_GRP,SKU_GROUP,IN_SKU_GROUP,TOTAL_ORDER_QUANTITY,TOTAL_MISS_QTY,TOTAL_UFR_QUANTITY,REMAINING_MISS_QTY,FILL_RATE_BEFORE,FILL_RATE_AFTER,UFR_CHG_VAL
213,155,single,"['DCF809', 'DCF840', 'DCF885']",1,204816,22013,182803,11694,0.89252,0.94290,5.64487
217,155,single,"['DCF809', 'DCF840']",1,203997,21564,182433,11326,0.89429,0.94448,5.61192
211,155,single,"['DCF809', 'DCF840', 'DCF885', 'DCF887']",1,207974,23450,184524,13131,0.88725,0.93686,5.59223
215,155,single,"['DCF809', 'DCF840', 'DCF887']",1,207155,23001,184154,12763,0.88897,0.93839,5.55948
181,155,single,"['DCF787', 'DCF809', 'DCF840', 'DCF885']",1,239108,24445,214663,13116,0.89777,0.94515,5.27757
185,155,single,"['DCF787', 'DCF809', 'DCF840']",1,238289,23996,214293,12748,0.89930,0.94650,5.24889
183,155,single,"['DCF787', 'DCF809', 'DCF840', 'DCF887']",1,241447,25433,216014,14185,0.89466,0.94125,5.20707
